In [ ]:
import ast
import os
import re
import subprocess
import tango
from dataclasses import dataclass, field
from datetime import datetime, timezone
from typing import Tuple, List, Match
from ska_mid_jupyter_notebooks.helpers.configuration import get_dish_namespace

# Regex patterns - compiled for reuse efficiency
INCOMING_COMMAND_CALL_REGEX_PATTERN = re.compile(r"-> (\w+\.\w+)\(\)")
RETURN_COMMAND_CALL_REGEX_PATTERN = re.compile(r"^(.*) <- (\w+\.\w+)\(\)")
LRC_RETURN_VAL_REGEX_PATTERN = re.compile(r"\(\[(.*)\], \['(.*)'\]\)")
LRC_TUPLE_REGEX_PATTERN = re.compile(r"'([0-9a-zA-Z._]*)', '([^']*)'")
LOG_REGEX_PATTERN = re.compile(r"([^|]*)\|([^|]*)\|([^|]*)\|([^|]*)\|([^|]*)\|([^|]*)\|([^|]*)\|(.*)")
EVENT_REGEX_PATTERN = re.compile(r"([^\t]*)\t([^\t]*)\t([^\t]*)\t(.*)")
LOG_LRC_RESULT_REGEX_PATTERN = re.compile(r"Received longRunningCommandResult event for\s*device:?\s*(.*?),?\s*with value:\s*\('(.*?)',\s*'\[\d,\s*\"(.*?)\"\]'\)")
INVOKE_EXECUTE_COMMAND_REGEX_PATTERN = re.compile(r"(?:Invoked|About to execute command) \[?(.*?)\]? on device (\[.*?\]|[^\[]+)")
DEBUG_PATCH_FORWARD_REGEX_PATTERN = re.compile(r"->\s*(\w+)\.(\w+)\(\)")
K_VALUES_TO_DISH_REGEX_PATTERN = re.compile(r"Invoking (.*) on dish adapter (.*)")
CSP_SDP_ON_COMMAND_REGEX_PATTERN = re.compile(r"Invoking (.*) command for (.*) devices\s*")

# Limits for spammy calls
TRACK_LOAD_TABLE_LIMIT = 8

# Log time adjustment due to delays
GENERAL_LOG_TIME_ADJUSTMENT_SECONDS = 0.08
DEBUG_LOG_TIME_ADJUSTMENT_SECONDS = 0.25

### Define tracked devices and pods

In [ ]:
# Define dish indexes to use
dish_indexes = ['001', '063']

# Define namespaces
sut_namespace = 'staging'
dish_namespaces = [get_dish_namespace(sut_namespace, f'SKA{index}') for index in dish_indexes]

# Define tango hosts
sut_tango_host = f'tango-databaseds.{sut_namespace}.svc.miditf.internal.skao.int:10000'
dish_tango_hosts = [
    f'tango-databaseds.{dish_namespace}.svc.miditf.internal.skao.int:10000'
    for dish_namespace in dish_namespaces
]

# Define device TRLs (for events)
tracked_device_trls = [
    f'{sut_tango_host}/ska_mid/tm_central/central_node',
    f'{sut_tango_host}/ska_mid/tm_subarray_node/1',
    f'{sut_tango_host}/ska_mid/tm_leaf_node/csp_master',
    f'{sut_tango_host}/ska_mid/tm_leaf_node/csp_subarray01',
    f'{sut_tango_host}/mid-csp/subarray/01',
#    f'{sut_tango_host}/mid_csp_cbf/sub_elt/subarray_01',
#    f'{sut_tango_host}/mid-sdp/subarray/01',
]

tracked_device_trls.extend(
    f'{sut_tango_host}/ska_mid/tm_leaf_node/ska{index}'
    for index in dish_indexes
)

tracked_device_trls.extend(
    f'{dish_host}/mid-dish/dish-manager/ska{index}'
    for dish_host, index in zip(dish_tango_hosts, dish_indexes)
)

tracked_device_trls.extend(
    f'{dish_host}/mid-dish/ds-manager/ska{index}'
    for dish_host, index in zip(dish_tango_hosts, dish_indexes)
)

# Define pods to get logs from
sut_device_pods = [
    'ds-centralnode-01-0',
    'ds-subarraynode-01-0',
    'ds-cspmasterleafnode-01-0',
    'ds-cspsubarray-staging-subarray1-0',
    'ds-cspsubarrayleafnode-01-0',
]
sut_device_pods.extend(f'ds-dishleafnode-{index}-0' for index in dish_indexes)

dish_device_pods = [
    [f'ds-dishmanager-{index}-0', f'ds-dsmanager-{index}-0',]
    for index in dish_indexes
]

# Dictionary containing namespaces as keys and corresponding pods as values
namespaces_pods = {
    sut_namespace: sut_device_pods,
}
namespaces_pods.update({
    dish_namespace: dish_pods
    for dish_namespace, dish_pods in zip(dish_namespaces, dish_device_pods)
})

### Set up device hierarchies

In [ ]:
# Declare likely callers for each device
device_hierarchy = [
    ['notebook', 'tm_central.central_node', 'tm_leaf_node.csp_master', 'mid-csp.control.0'],
    ['tm_central.central_node', 'tm_leaf_node.sdp_master'],
    ['tm_central.central_node', 'tm_subarray_node.2'],
    ['tm_central.central_node', 'tm_subarray_node.1'],
    ['tm_subarray_node.1', 'tm_leaf_node.sdp_subarray01'], # 'mid-sdp.subarray.01'],
    ['tm_subarray_node.1', 'tm_leaf_node.csp_subarray01', 'mid-csp.subarray.01'],
]

# Use a for loop because the dish devices should be grouped for the diagram
for index in dish_indexes:
    device_hierarchy.append([
        'tm_subarray_node.1', f'tm_leaf_node.d0{index}', f'dish-manager.ska{index}', f'ds-manager.ska{index}'
    ])

    device_hierarchy.append([
        f'dish-manager.ska{index}', f'ska{index}.spfrxpu.controller'
    ])
    
    device_hierarchy.append([
        f'dish-manager.ska{index}', f'simulator_spfc.ska{index}'
    ])


### PlantUML helper class

In [ ]:
class PlantUMLSequenceDiagram:
    def __init__(self):
        self.diagram_code = ""

    def start_diagram(self, title, actor):
        self.diagram_code = "@startuml SequenceDiagram\n"
        self.diagram_code += f"title {title}\n"
        self.diagram_code += f"actor {actor}\n"

    def add_participant(self, participant):
        self.diagram_code += f"participant {self.clean_text(participant)}\n"

    def end_diagram(self):
        self.diagram_code += "@enduml"

    def clean_text(self, text):
        return text.replace("-", "_")

    def wrap_text(self, text, max_width=50, max_length=500):
        if max_width <= 0:
            raise ValueError("max_width must be a positive integer.")

        if len(text) > max_length:
            truncated_length = max_length - 3
            text = text[:truncated_length] + "..."

        lines = []
        for i in range(0, len(text), max_width):
            end_index = i + max_width
            segment = text[i:end_index]
            lines.append(segment)

        result = "\n".join(lines).encode("unicode_escape").decode("utf-8")
        return result

    def wrap_text_on_spaces(self, text, max_width=50):
        if max_width <= 0:
            raise ValueError("max_width must be a positive integer.")

        words = text.split(" ")
        line = ""
        lines = []

        for word in words:
            if len(line) + len(word) < max_width:
                line += " " + word
            else:
                lines.append(line)
                line = word
        lines.append(line)

        result = "\n".join(lines).encode("unicode_escape").decode("utf-8")
        return result

    def add_note_over(self, device, note, color="lightgreen"):
        device = self.clean_text(device)
        # note = self.wrap_text(note)

        self.diagram_code += f"rnote over {device} #{color}: {note}\n"

    def add_hexagon_note_over(self, device, note, color="lightgrey"):
        device = self.clean_text(device)
        # note = self.wrap_text(note)

        self.diagram_code += f"hnote over {device} #{color}: {note}\n"

    def add_command_call(self, from_device, to_device, note):
        from_device = self.clean_text(from_device)
        to_device = self.clean_text(to_device)
        note = self.wrap_text(note)

        self.diagram_code += f"{from_device} -> {to_device}: {note}\n"

    def add_command_response(self, from_device, to_device, note):
        from_device = self.clean_text(from_device)
        to_device = self.clean_text(to_device)
        note = self.wrap_text(note)

        self.diagram_code += f"{from_device} --> {to_device}: {note}\n"

### Log parser class

In [ ]:
class LogParser:
    def __init__(self):
        self.log_pattern_callbacks: list[tuple[str, callable]] = []

    def _parse_log_line(self, log_line):
        for pattern, pattern_cb in self.log_pattern_callbacks:
            match = re.search(pattern, log_line)
            if match:
                # prefix|iso_date_string|log_level|runner|action|log_line|device|message
                group_values = match.groups()
                pattern_cb(*group_values)
                break

    def parse_file(self, file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            logs = file.readlines()

        for log in logs:
            self._parse_log_line(log)

### Custom event and log parser class

In [ ]:
class EventsAndLogsFileParser(LogParser):
    def __init__(self, limit_track_load_table_calls=True, show_events=False,  show_component_state_updates=False, device_hierarchy: list=[]):
        super().__init__()

        self.limit_track_load_table_calls = limit_track_load_table_calls
        self.show_events = show_events
        self.show_component_state_updates = show_component_state_updates

        self.sequence_diagram = PlantUMLSequenceDiagram()
        self.device_hierarchy = device_hierarchy
        self.running_lrc_status_updates = {}

        self.log_pattern_callbacks = [
            [LOG_REGEX_PATTERN, self.log_callback],
            [EVENT_REGEX_PATTERN, self.event_callback],
        ]

        self.track_load_table_count = 0

    def get_likely_caller_from_hierarchy(self, device) -> str:
        for hierarchy_list in self.device_hierarchy:
            if device not in hierarchy_list or device == hierarchy_list[0]:
                continue
            device_index = hierarchy_list.index(device)
            hierarchy_index = self.device_hierarchy.index(hierarchy_list)
            likely_caller = self.device_hierarchy[hierarchy_index][device_index - 1]
            # print(f'Likely caller of device {device} is {likely_caller}')
            return likely_caller
        print(f'Setting unknown caller for device {device}')
        return "unknown"

    def get_method_from_lrc_id(self, lrc_id) -> str:
        return "_".join(lrc_id.split("_")[2:])

    def parse(self, file_path: str, output_file_path: str, actor="pytest"):
        log_file_name = file_path.split("/")[-1]

        cleaned_log_file_name = self.sequence_diagram.clean_text(log_file_name)
        title = f"Sequence diagram generated from\n{cleaned_log_file_name}".encode(
            "unicode_escape"
        ).decode("utf-8")

        self.running_lrc_status_updates = {}
        self.sequence_diagram.start_diagram(title, actor)

        # Add participants to ensure order of swimlanes
        for hierarchy_list in self.device_hierarchy:
            for device in hierarchy_list[1:]:
                self.sequence_diagram.add_participant(device)

        self.parse_file(file_path)

        self.sequence_diagram.end_diagram()

        # Save the PlantUML diagram code to a file
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(self.sequence_diagram.diagram_code)

    def get_cleaned_device_name(self, device: str, info_type: str = 'none') -> str:
        cleaned_device = device.split('/', maxsplit=1)[1]  # e.g. dish-manager/ska001

        # mid-csp and mid-sdp are both just called "subarray" or "control", to differentiate, add the 
        # first part of the trl. The spfrxpu devices also need the first element to get the dish number
        if cleaned_device.startswith('subarray/') \
            or cleaned_device.startswith('spfrxpu/') \
            or cleaned_device.startswith('control/'):
            if 'event' in info_type:
                # e.g. MidCspSubarray(mid-csp/subarray/01)
                trl_start = device.split('(', 1)[1]
            elif 'log' in info_type:
                # e.g. tango-device:mid-csp/subarray/01
                trl_start = device.split('tango-device:', 1)[1]
            else:
                # e.g. mid-csp/subarray/01 or [ska001/spfrxpu/controller]
                trl_start = device.strip('[')

            cleaned_device = f'{trl_start.split("/")[0]}/{cleaned_device}'

        # Remove last character if it's an event because the closing parenthesis will still be there
        cleaned_device = cleaned_device[:-1] if info_type == 'event' or device.startswith('[') else cleaned_device

        # Replace / with . and for plantUML
        cleaned_device = cleaned_device.replace('/', '.')

        return cleaned_device.lower()

    def log_callback(self, prefix: str, iso_date_string: str, log_level: str,
                     runner: str, action: str, log_line: str, device: str, message: str):
        # Ignore empty devices        
        if device == "":
            return
        
        # Example log message:
        # 1724676115.079 -  Log  - 1|2024-08-26T12:41:55.079Z|DEBUG|Thread-9 (_event_consumer)|
        # _component_state_changed|dish_manager_cm.py#390|tango-device:mid-dish/dish-manager/SKA001|...
        cleaned_device = self.get_cleaned_device_name(device, 'log')

        if log_level.strip() == 'INFO' and action in ['update_long_running_command_result', 'update_command_result']:
            # <prefix>|<date>|INFO|longRunningCommandResult|update_long_running_command_result|<log_line>|
            # tango-device:ska_mid/tm_subarray_node/1|Received longRunningCommandResult event for device:
            #  ska_mid/tm_leaf_node/sdp_subarray01, with value: ('1731055110.2204533_15076717473253_On', '[0, "Command Completed"]')
            self.handle_lrc_result_log(cleaned_device, message)

        elif log_level.strip() == 'DEBUG' and action in ['invoke_command', 'execute_command']:
            # <prefix>|<date>|DEBUG|<runner>|invoke_command|<log_line>|tango-device:ska_mid/tm_central/central_node|
            # Invoked On on device ska_mid/tm_subarray_node/2
            self.handle_invoke_or_execute_command_log(cleaned_device, message)

        elif action == '_debug_patch':
            # <prefix>|<date>|DEBUG|<runner|_debug_patch|<log_line>|tango-device:ska_mid/tm_central/central_node|
            # -> CentralNodeMid.TelescopeOn()
            self.handle_debug_patch_log(cleaned_device, message)

        elif action == '_set_k_numbers_to_dish':
            # <prefix>|<date>|INFO|<runner>|_set_k_numbers_to_dish|<log_line>|
            # tango-device:ska_mid/tm_central/central_node|Invoking SetKValue on dish adapter ska_mid/tm_leaf_node/d0001
            self.handle_set_k_numbers_to_dish_logs(cleaned_device, message)
        
        elif action in ['turn_on_csp', 'turn_on_sdp']:
            # <prefix>|<date>|INFO|<runner>|turn_on_csp|<log_line>|tango-device:ska_mid/tm_central/central_node|
            # Invoking On command for ska_mid/tm_leaf_node/csp_master devices
            self.handle_csp_sdp_devices_on_command_log(cleaned_device, message)

        elif action == "_info_patch":
            self.info_patch_cb(
                prefix, iso_date_string, log_level, runner, action, log_line, cleaned_device, message
            )
        elif action == "_update_component_state" and self.show_component_state_updates:
            self.component_state_update_cb(
                prefix, iso_date_string, log_level, runner, action, log_line, cleaned_device, message
            )

    # Helper method for handling longRunningCommandResult logs
    def handle_lrc_result_log(self, cleaned_device: str, message: str):
        """Handles parsing of longRunningCommandResult logs and updates the sequence diagram."""
        match = LOG_LRC_RESULT_REGEX_PATTERN.search(message)
        if match:
            from_device = match.group(1).strip()
            command_id = match.group(2).strip()
            status = match.group(3).strip()

            # Ignore staging statuses for cleaner diagrams
            if status.lower() == 'staging':
                return

            # Remove ID from command 
            # 1731336386.5340867_71131397947360_LoadDishCfg to LoadDishCfg
            command = command_id.split('_')[-1]

            # Clean the target device name for PlantUML
            cleaned_from_device = self.get_cleaned_device_name(from_device)

            # Add an arrow to the sequence diagram
            self.sequence_diagram.add_command_response(
                cleaned_from_device, cleaned_device, f'""{command}"" -> {status}'
            )

    # Helper method for handling invoke_command and execute_command logs
    def handle_invoke_or_execute_command_log(self, cleaned_device: str, message: str):
        """Handles parsing of invoke_command and execute_command logs and updates the sequence diagram."""
        match = INVOKE_EXECUTE_COMMAND_REGEX_PATTERN.search(message)
        if match:
            command_name = match.group(1).strip()

            # Exclude spammy commands from the diagram
            if command_name in ["MonitorPing", "TrackLoadTable"]:
                return
            
            self.generic_command_match_handling(match, cleaned_device)

    # Helper method for handling forward direction _debug_patch logs
    def handle_debug_patch_log(self, cleaned_device: str, message: str):
        """Handles parsing of _debug_patch logs and updates the sequence diagram."""
        match = DEBUG_PATCH_FORWARD_REGEX_PATTERN.search(message)
        if match:
            target_class = match.group(1)
            command_name = match.group(2)

            # Use the device hierarchy to get the likely caller
            likely_caller = self.get_likely_caller_from_hierarchy(cleaned_device)

            # Most of these logs are directly called from the notebook
            # Small trick for setting the notebook as the caller for all mid subarray calls
            if target_class == "SubarrayNodeMid":
                likely_caller = self.get_likely_caller_from_hierarchy(likely_caller)

            # Add an arrow to the sequence diagram from the likely caller to the target device
            self.sequence_diagram.add_command_call(
                likely_caller, cleaned_device, f'""{target_class}.{command_name}""'
            )

    # Helper method for handling _set_k_numbers_to_dish logs
    def handle_set_k_numbers_to_dish_logs(self, cleaned_device: str, message: str):
        """Handles parsing of _set_k_numbers_to_dish logs and updates the sequence diagram."""
        match = K_VALUES_TO_DISH_REGEX_PATTERN.search(message)
        if match:
            self.generic_command_match_handling(match, cleaned_device)

    # Helper method for handling turn_on_csp and turn_on_sdp devices logs
    def handle_csp_sdp_devices_on_command_log(self, cleaned_device: str, message: str):
        """Handles parsing turn_on_csp and turn_on_sdp logs and updates the sequence diagram"""
        # this is not "On" for the master device, but to get them to turn on the lower devices
        # Adjusting the text of the command going to the sequence diagram for clarity
        adjustment = 'On for lower devices'
        message = message.replace('On', adjustment)

        match = CSP_SDP_ON_COMMAND_REGEX_PATTERN.search(message)
        if match:
            self.generic_command_match_handling(match, cleaned_device)

    def info_patch_cb(self, prefix, iso_date_string, log_level, runner,
                      action, log_line, device, message):
        if "->" in message:
            match = INCOMING_COMMAND_CALL_REGEX_PATTERN.search(message)
            if match:
                method = match.group(1)
                method = method.split(".")[1]

                # Reduce "TrackLoadTable" commands from the diagram
                if method == "TrackLoadTable":
                    self.track_load_table_count += 1
                    if self.limit_track_load_table_calls and \
                       self.track_load_table_count > TRACK_LOAD_TABLE_LIMIT:
                        return

                caller = self.get_likely_caller_from_hierarchy(device)
                self.sequence_diagram.add_command_call(caller, device, f'""{method}""')

        elif "<-" in message:
            match = RETURN_COMMAND_CALL_REGEX_PATTERN.search(message)
            if match:
                return_val = match.group(1)
                method = match.group(2)
                method = method.split(".")[1]

                # Reduce "TrackLoadTable" commands from the diagram
                if method == "TrackLoadTable":
                    self.track_load_table_count += 1
                    if self.limit_track_load_table_calls and \
                       self.track_load_table_count > TRACK_LOAD_TABLE_LIMIT:
                        return

                caller = self.get_likely_caller_from_hierarchy(device)

                self.sequence_diagram.add_command_response(
                    device, caller, f'""{method}"" -> {return_val}'
                )

    def component_state_update_cb(self, prefix, iso_date_string, log_level,
                                  runner, action, log_line, device, message):
        search_string = r"Updating (\w*) (\w*) component state with \[(.*)\]"

        match = re.search(search_string, message)

        if match:
            string_dict = match.group(3)
            string_dict = string_dict.replace("<", "'<")
            string_dict = string_dict.replace(">", ">'")
            component_state_updates = ast.literal_eval(string_dict)

            note_text = "Component state update"
            for attr, attr_value in component_state_updates.items():
                note_text += f'\n""{attr} = {attr_value}""'.encode("unicode_escape").decode(
                    "utf-8"
                )
            self.sequence_diagram.add_hexagon_note_over(device, note_text)

    def generic_command_match_handling(self, match: Match[str], cleaned_device: str):
        """Handle a generic command with target device and add it to the sequence diagram"""
        command_name = match.group(1).strip()
        target_device = match.group(2).strip()

        # Clean the target device name for PlantUML
        cleaned_target_device = self.get_cleaned_device_name(target_device)

        # Add an arrow to the sequence diagram
        self.sequence_diagram.add_command_call(
            cleaned_device, cleaned_target_device, f'""{command_name}""'
        )

    def event_callback(self, prefix, device: str, event_attr, val):
        # 1724660914.761 - Event - 2024-08-26 08:28:34.761448	DishManager(mid-dish/dish-manager/ska001)
        # 	longrunningcommandstatus	('1724660914.663982_241979260268973_SetStowMode', 'COMPLETED')
        # 1724660914.761 - Event - 2024-09-18 08:44:43.859312	MidCspSubarray(mid-csp/subarray/01)
        #   longrunningcommandstatus	('1726641882.6817706_174896405886953_AssignResources', 'STAGING')
        cleaned_device = self.get_cleaned_device_name(device, "event")
        caller = self.get_likely_caller_from_hierarchy(cleaned_device)

        if "longrunningcommand" in event_attr:
            self.handle_lrc_event_log(cleaned_device, caller, event_attr, val)
        elif self.show_events:
            self.sequence_diagram.add_note_over(
                cleaned_device,
                f'Event\n""{event_attr} = {val.strip()}""'.encode("unicode_escape").decode(
                    "utf-8"
                ),
            )

    def handle_lrc_event_log(self, device, caller, event_attr, val):
        if "longrunningcommandstatus" in event_attr:
            lrc_statuses = LRC_TUPLE_REGEX_PATTERN.findall(val)
            for index, (lrc_id, status) in enumerate(lrc_statuses):
                # If there are any newer updates for this lrc in the LRC statuses then skip this
                newer_status_found = False
                if index + 1 < len(lrc_statuses):
                    for i in range(index + 1, len(lrc_statuses)):
                        if lrc_statuses[i][0] == lrc_id:
                            newer_status_found = True
                            break

                if newer_status_found:
                    break

                method_name = self.get_method_from_lrc_id(lrc_id)

                if status == "STAGING":
                    # Only track methods which are called in the scope of the file
                    # This avoids some noise left over in LRC attributes from previous test / setup
                    self.running_lrc_status_updates[lrc_id] = []

                # Only update if its a method called in the scope of this file and its a new status
                if (
                    lrc_id in self.running_lrc_status_updates
                    and status not in self.running_lrc_status_updates[lrc_id]
                ):
                    self.running_lrc_status_updates[lrc_id].append(status)
                    self.sequence_diagram.add_command_response(
                        device, caller, f'""{method_name}"" -> {status}'
                    )
        elif "longrunningcommandprogress" in event_attr:
            lrc_progresses = LRC_TUPLE_REGEX_PATTERN.findall(val)
            for lrc_id, progress in lrc_progresses:
                # Only show progress updates for methods which have been staged
                if lrc_id in self.running_lrc_status_updates:
                    method_name = self.get_method_from_lrc_id(lrc_id)
                    self.sequence_diagram.add_command_call(
                        device, device, f'""{method_name}"" -> {progress}'
                    )
        elif event_attr == "longrunningcommandresult":
            pass

### Setup tracked devices

In [14]:
@dataclass
class TrackedDevice:
    """Class to group tracked device information"""

    device_proxy: tango.DeviceProxy
    attribute_names: Tuple[str]
    subscription_ids: List[int] = field(default_factory=list)


tracked_devices = [
    TrackedDevice(
        tango.DeviceProxy(device_trl),
        (
            "longrunningcommandstatus",
            "longrunningcommandresult",
            "longrunningcommandprogress",
        ),
    )
    for device_trl in tracked_device_trls
]

### Event printer class

In [15]:
class EventPrinter:
    """Class that writes attribute changes to a file"""

    def __init__(self, filename: str, tracked_devices: Tuple[TrackedDevice] = ()) -> None:
        self.tracked_devices = tracked_devices
        self.filename = filename
        self.events = []

    def __enter__(self):
        for tracked_device in self.tracked_devices:
            dp = tracked_device.device_proxy
            for attr_name in tracked_device.attribute_names:
                sub_id = dp.subscribe_event(attr_name, tango.EventType.CHANGE_EVENT, self)
                tracked_device.subscription_ids.append(sub_id)

    def __exit__(self, exc_type, exc_value, exc_tb):
        for tracked_device in self.tracked_devices:
            try:
                dp = tracked_device.device_proxy
                for sub_id in tracked_device.subscription_ids:
                    dp.unsubscribe_event(sub_id)
            except tango.DevError:
                pass

    def add_event(self, timestamp, message):
        self.events.append((timestamp, message))
        with open(self.filename, "a") as open_file:
            open_file.write("\n" + message)

    def push_event(self, ev: tango.EventData):
        event_string = ""
        if ev.err:
            err = ev.errors[0]
            event_string = f"\nEvent Error {err.desc} {err.origin} {err.reason}"
        else:
            attr_name = ev.attr_name.split("/")[-1]
            attr_value = ev.attr_value.value
            if ev.attr_value.type == tango.CmdArgType.DevEnum:
                attr_value = ev.device.get_attribute_config(attr_name).enum_labels[attr_value]

            event_string = f"Event - {ev.reception_date}\t{ev.device}\t{attr_name}\t{attr_value}"

        self.add_event(ev.reception_date.totime(), event_string)

### Instantiate the event printer

In [ ]:
os.environ["TZ"] = "Africa/Johannesburg"

# Get the current datetime
datetime_start = datetime.now(timezone.utc) 
iso_start = datetime_start.isoformat()

date = datetime_start.strftime("%Y%m%d")
time_start = datetime_start.strftime("%H%M%S")
events_file_name = f"generated_events-{date}-{time_start}.txt"
event_printer = EventPrinter(
    events_file_name, tracked_devices
)

### Start the event printer monitoring before running commands on your monitored device

In [ ]:
event_printer.__enter__()

### Exit the printer to unsubscribe from attributes

In [ ]:
event_printer.__exit__(None, None, None)

### Function to retrieve logs from pods

In [ ]:
all_pod_logs = {}

def get_iso_date_string_from_string(val: str):
    # Log example: 1|2024-11-08T08:38:30.211Z|INFO|...
    iso_date_string_pattern = r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z"
    match = re.search(iso_date_string_pattern, val)
    return match.group() if match else ""

def get_pod_logs_and_timestamps(namespace, pod_name, since_time):
    command = f"kubectl logs {pod_name} -n {namespace} --since-time={since_time}"
    # print(command)

    try:
        # Run the command and capture the output
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        api_response = result.stdout  # Capture standard output

        # Check if there is any error output (0 is success)
        if result.returncode != 0:
            print(f"Error for {pod_name}: {result.stderr}")
            return []
        
        # Make sure api_response is not empty
        if not api_response:
            print(f"No logs found for {pod_name}.")
            return []
        
        with open(f'{pod_name}-{date}-{time_start}.txt', 'w', encoding='utf-8') as f:
            f.write(api_response)

        logs = api_response.splitlines()
    except Exception as e:
        print(f"An error occurred while running the command for {pod_name}: {e}")
        return []

    # Extract times from each line in the logs (e.g. 2024-08-26T07:16:11.051Z)
    extracted_logs = []
    for log in logs:
        # Remove null characters from the line
        log = log.replace("\x00", "").strip()
        # Skip empty logs
        if log == "":
            continue
        
        # Search for the timestamp in each log entry
        iso_date_string = get_iso_date_string_from_string(log)
        if iso_date_string:
            try:
                # Parse the timestamp and add it to the extracted logs
                time_obj = datetime.strptime(iso_date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
                adjusted_timestamp = time_obj.timestamp()
                if '|DEBUG|' in log:
                    adjusted_timestamp -= DEBUG_LOG_TIME_ADJUSTMENT_SECONDS
                else:
                    adjusted_timestamp -= GENERAL_LOG_TIME_ADJUSTMENT_SECONDS

                extracted_logs.append((adjusted_timestamp, f" Log  - {log}"))
            except ValueError as e:
                print(f"Timestamp parsing error for log: {log} - {e}")
        else:
            print(f"No timestamp found in log: {log}")

    return extracted_logs

### Parse the log files

In [ ]:
events_and_logs_file_name = f"events_and_logs-{date}-{time_start}.txt"
sequence_diagram_file_name = f"sequence-diagram-{date}-{time_start}.puml"

# Loop over each namespace and its pods
for namespace, pods in namespaces_pods.items():
    for pod in pods:
        all_pod_logs[pod] = get_pod_logs_and_timestamps(namespace, pod, iso_start)


No timestamp found in log: Device states before executing TelescopeOn command
No timestamp found in log: Devices  STATE
No timestamp found in log: 0                                                                                         ska_mid/tm_subarray_node/1      0
No timestamp found in log: 1                                                                                         ska_mid/tm_subarray_node/2      0
No timestamp found in log: 2                                                                                ska_mid/tm_leaf_node/csp_subarray01      0
No timestamp found in log: 3                                                                                ska_mid/tm_leaf_node/csp_subarray02      0
No timestamp found in log: 4                                                                                ska_mid/tm_leaf_node/sdp_subarray01      0
No timestamp found in log: 5                                                                                ska_mid/tm_leaf_n

In [ ]:
# # TESTING
# test_pods = ['ds-centralnode-01-0',
#              'ds-cspmasterleafnode-01-0',
#              'ds-cspsubarray-sut-subarray1-0',
#              'ds-cspsubarray1-0',
#              'ds-dishleafnode-001-0',
#              'ds-dishleafnode-036-0',
#              'ds-dishmanager-001-0',
#              'ds-dishmanager-036-0',
#              'ds-dsmanager-001-0',
#              'ds-dsmanager-036-0',
#              'ds-subarraynode-01-0',
#              ]
# logs = []

# for test_pod in test_pods:
#     with open(f'{test_pod}.txt', 'r', encoding='utf-8', errors='ignore') as f:
#         contents = f.read()
#     logs = contents.splitlines()

#     extracted_logs = []
#     for log in logs:
#         # Remove null characters from the line
#         log = log.replace("\x00", "").strip()
#         # Skip empty logs
#         if log == "":
#             continue
        
#         # Search for the timestamp in each log entry
#         iso_date_string = get_iso_date_string_from_string(log)
#         if iso_date_string:
#             try:
#                 # Parse the timestamp and add it to the extracted logs
#                 time_obj = datetime.strptime(iso_date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
#                 adjusted_timestamp = time_obj.timestamp()
#                 if '|DEBUG|' in log:
#                     adjusted_timestamp -= DEBUG_LOG_TIME_ADJUSTMENT_SECONDS
#                 else:
#                     adjusted_timestamp -= GENERAL_LOG_TIME_ADJUSTMENT_SECONDS

#                 extracted_logs.append((adjusted_timestamp, f" Log  - {log}"))
#             except ValueError as e:
#                 print(f"Timestamp parsing error for log: {log} - {e}")
#         else:
#             continue

#         all_pod_logs[test_pod] = extracted_logs

### Combine events and logs into a file

In [ ]:
# Combine and sort logs/events
captured_events = event_printer.events
combined_events_and_logs = []

for logs in all_pod_logs.values():
    combined_events_and_logs.extend(logs)

combined_events_and_logs.extend(captured_events)

combined_events_and_logs.sort(key=lambda x: x[0])

# print(combined_events_and_logs)

# Write the combined entries to the output file
with open(events_and_logs_file_name, "w") as file:
    for timestamp, message in combined_events_and_logs:
        file.write(f"{timestamp:.3f} - {message}\n")

### Parse the events and logs files

In [ ]:
# To generate a more verbose diagram with events and component state updates,
# set first two arguments to True
file_parser = EventsAndLogsFileParser(
    limit_track_load_table_calls=True,
    show_events=False,
    show_component_state_updates=False,
    device_hierarchy=device_hierarchy,
)

events_and_logs_file_path = f"./{events_and_logs_file_name}"

file_parser.parse(
    events_and_logs_file_path, sequence_diagram_file_name, actor="notebook"
)